In [1]:
from nltk.corpus import stopwords
import pandas as pd
import re

In [2]:
data= pd.read_csv('result.csv')
data.head()

,timestamp,tweet_text
0,2021-12-09 17:00:12+00:00,Today we shared details about how @CVSHealth i...
1,2021-12-02 16:38:09+00:00,It was a pleasure to join @Microsoft CEO @Saty...
2,2021-12-01 14:29:59+00:00,Congratulations to this year’s #Bloomberg50. I...
3,2021-11-11 14:00:19+00:00,"On #VeteransDay, I want to recognize and thank..."
4,2021-11-03 13:15:11+00:00,Earlier today @CVSHealth posted third quarter ...


In [3]:
stop = stopwords.words('english')

In [4]:
df = data.copy()

In [5]:
def remove_URL(text):
    
    return re.sub(r'https?://\S+|www\.\S+', r'',text)

In [6]:
df['tweet_text']=df['tweet_text'].apply(lambda x : remove_URL(x))
df

,timestamp,tweet_text
0,2021-12-09 17:00:12+00:00,Today we shared details about how @CVSHealth i...
1,2021-12-02 16:38:09+00:00,It was a pleasure to join @Microsoft CEO @Saty...
2,2021-12-01 14:29:59+00:00,Congratulations to this year’s #Bloomberg50. I...
3,2021-11-11 14:00:19+00:00,"On #VeteransDay, I want to recognize and thank..."
4,2021-11-03 13:15:11+00:00,Earlier today @CVSHealth posted third quarter ...
...,...,...
829,2017-08-24 15:05:29+00:00,Thank you @GovMattBevin for being an ally in t...
830,2017-08-23 22:00:17+00:00,"With @AbleTo_Health, we provide consumers with..."
831,2017-08-23 20:57:36+00:00,RT @GovMattBevin: We don't have the luxury of ...
832,2017-08-23 20:16:12+00:00,Proud of our efforts to combat the opioid epid...
